In [ ]:
pip install folium

In [ ]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=45763f8525124e994779e8cd2ebd4c1a98163ef7987314e07d3ec04a359e9754
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import pandas as pd
import numpy as np
import folium
import wget
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [ ]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file)

In [ ]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

In [ ]:
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

In [ ]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [ ]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#ff0000', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#ff0000;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for index, site in launch_sites_df.iterrows():
    location = [site['Lat'], site['Long']]
    circle = folium.Circle(location, radius=50, color='#ff0000', fill=True).add_child(folium.Popup(site['Launch Site']))
    marker = folium.map.Marker(location,
                               icon=DivIcon(
                                   icon_size=(20, 20),
                                   icon_anchor=(0, 0),
                                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
                               )
                               )
    site_map.add_child(circle)
    site_map.add_child(marker)

In [ ]:
marker_cluster = MarkerCluster()

In [ ]:
def assign_marker_color(launch_outcome):
    return 'green' if launch_outcome == 1 else 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

In [ ]:
for index, row in spacex_df.iterrows():
    folium.map.Marker(
        (row['Lat'], row['Long']),
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
    ).add_to(marker_cluster)

In [ ]:
site_map.add_child(marker_cluster)

In [ ]:
site_map